In [1]:
import pandas as pd
import requests
import csv
import time
from google.cloud import bigquery
import os

On blokker.nl, roughly 500,000 unique products are being sold. The vast majority (98-99%) of these products are pushed by our external sellers. Unfortunately, the categorization for these products is not always correct. In this notebook, we are recategorizing the best selling products from our external sellers with the help of AI. We are using ChatGPT with the gpt-4o model. The idea is to use a recursive API call, where each time a list of (sub)categories are sent to the ChatGPT API in combination with product name and product description.

Our first category level (L1) contains the categories below:
- Speelgoed
- Dieren
- Wonen
- Reizen
- Eten & tafelen
- Persoonlijke verzorging
- Schoonmaak & huishoud
- Elektronica
- Koken & bakken
- Tuin
- Baden & slapen
- Wassen & strijken
- Vrije tijd
- Baby
- Boeken & kantoorartikelen



In [2]:
%cd C:\Users\KoeReu\OneDrive - Blokker B.V\Bureaublad\Project hercategorisering
    
already_recategorized_ids = []
    
with open('recategorization_file.csv', 'r', encoding = 'utf-8') as f:
    csv_reader = csv.reader(f, delimiter = ';')
    
    # skip first line
    next(csv_reader)
    
    for row in csv_reader:
        already_recategorized_ids.append(row[0])

C:\Users\KoeReu\OneDrive - Blokker B.V\Bureaublad\Project hercategorisering


In [3]:
len(already_recategorized_ids)

1251

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:\\Users\\KoeReu\\OneDrive - Blokker B.V\\Bureaublad\\etc'

client = bigquery.Client()

In [5]:
# extract most sold products (based on unique order ID's) for products that are 
# not sold by Blokker. These are filtered out by the WHERE '' NOT IN UNNEST(arr) statement

# currently at lookback of 60 days

QUERY = (
    r"""
WITH cte AS (SELECT 
items.item_id AS item_id,
COUNT(DISTINCT(ecommerce.transaction_id)) AS order_q,
ARRAY_AGG(DISTINCT(items.affiliation)) AS arr
FROM `universal-analytics-nextail.analytics_283141299.events_*`, UNNEST(items) AS items
WHERE _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)) AND
FORMAT_DATE('%Y%m%d', CURRENT_DATE()) AND
event_name = 'purchase'
GROUP BY 1
ORDER BY 2)

SELECT cte.*, 
SUM(order_q) OVER(ORDER BY order_q ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumsum_order_q,
SUM(order_q) OVER(ORDER BY order_q ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) / SUM(order_q) OVER() AS cumsum_perc_order_q
FROM cte
WHERE '' NOT IN UNNEST(arr)
ORDER BY SUM(order_q) OVER(ORDER BY order_q ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) DESC
    """
        )

query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [8]:
bestseller_lst = []
bestseller_filter = []


for num, row in enumerate(rows):
    mdm_id, order_q, arr, cumsum_order_q, cumsum_perc_order_q = row
    bestseller_lst.append([num, mdm_id, order_q, arr, cumsum_order_q, cumsum_perc_order_q])
    bestseller_filter.append(mdm_id)

In [11]:
openai_secret_key = 'sk-proj-keykeykeykeykey'

headers = {"Authorization": f"Bearer {openai_secret_key}"}
api_url = 'https://api.openai.com/v1/chat/completions'

In [12]:
# read our product feed with specific columns
url = 'https://custom.channable.com/30325_blokker.csv'
usecols = ['mdm_productid', 'title', 'long_description', 'categoryPath', 'chunkname']
df = pd.read_csv(url, delimiter = '|', usecols = usecols, dtype = object, encoding = 'utf-8')

# drop rows with NaN values
df.dropna(inplace = True)

# create dataframe with unique chunk and category combinations
df_cats = df[['chunkname', 'categoryPath']].copy()
df_cats = df_cats.drop_duplicates().copy()
cats_lst = df_cats.values.tolist()

# remove all inductiebeschermers
df = df[df['chunkname'] != 'Inductiebeschermer'].copy()

# remove products that are not in the bestseller list
df = df[df['mdm_productid'].isin(bestseller_filter)].copy()
del(bestseller_filter)

# create product dict with key is product ID and values as product name, product description, chunk and category path
product_dict = {str(row[0]): list(row[1:]) for row in df.values}

In [13]:
# create chunk_dict with k: v as category: chunk
chunk_dict = {}

for row in cats_lst:
    chunk, cat = row
    chunk_dict[cat] = chunk

In [14]:
# remove categories with Acties
# this is used to create a nested dictionary with all categories
# e.g. {'Wonen': {'Meubelen', {'Banken', 'Stoelen'}, 'Woonaccesoires'}}
cats = list(set([row[-1] for row in cats_lst if row[-1].startswith('Acties') == False]))

In [15]:
def add_keys_nested_dict(d, keys):
    
    """This function creates a nested list of categories"""
    
    if len(keys) == 1:
        d.setdefault(keys[0], {})
    else:
        key = keys[0]
        if key not in d:
            d[key] = {}
        add_keys_nested_dict(d[key], keys[1:])

In [19]:
cat_dict.keys()

dict_keys(['Speelgoed', 'Dieren', 'Wonen', 'Reizen', 'Eten & tafelen', 'Persoonlijke verzorging', 'Schoonmaak & huishoud', 'Elektronica', 'Koken & bakken', 'Tuin', 'Baden & slapen', 'Wassen & strijken', 'Vrije tijd', 'Baby', 'Boeken & kantoorartikelen'])

In [16]:
# create nested cat_dict with the add_keys_nested_dict function above
cat_dict = {}

for cat in cats:
    cat = cat.split(' > ')
    add_keys_nested_dict(cat_dict, cat)

In [18]:
# before, the prompt contained
# Huidige categorie: {product_cat}

system_role = "Je bent een assistent die helpt bij het bepalen van een productcategorie uit een lijst van bestaande productcategorieën. Lever slechts één categorie terug uit de aangeleverde lijst!"

def create_prompt(product_title, product_desc, product_cat, d):
    
    
    prompt = f"""Plaats onderstaand artikel in één van onderstaande productcategorieën:
                Producttitel: {product_title}
                Productbeschrijving: {product_desc}
                Productcategorieën: {chr(10).join(d.keys())}.
    
                Lever alleen één volledige en exact overeenkomende productcategorie terug zonder bijbehorende tekst- of leestekens, zoals bijvoorbeeld Schoonmaak & huishoud of Wonen. 
                Als je geen goede categorie kan vinden, lever dan alleen het woord "stop" terug"""
    messages = [{"role": "system", "content": system_role}, {"role": "user", "content": prompt}]
    data = {"model": "gpt-4o", "messages": messages, 'max_tokens': 1000, 'temperature': 0.05}

    return data

In [16]:
def call_api(api_payload: dict):
    
    """ This function calls the ChatGPT API. Some very basic rate handling is being implemented
    but this definitely needs some improvement! """
    
    response = requests.post(api_url, headers = headers, json = api_payload)
    time.sleep(1)

    
    reset_tokens = response.headers['x-ratelimit-reset-tokens'].split('.')[0]
    reset_tokens = reset_tokens.replace('s', '')
    reset_tokens = int(reset_tokens)
    
    if reset_tokens > 20:
        print(f"x-ratelimit-remaining-requests: {response.headers['x-ratelimit-remaining-requests']}")
        time.sleep(reset_tokens)
        print(f"Slept for {reset_tokens} seconds (RT)")
    
    
    if response.json().get('error', False):
        print(response.json())
        time.sleep(15)
        call_api(api_payload)
        print('Slept for 15 seconds')
        
    new_cat = response.json()['choices'][0]['message']['content']
    return new_cat

In [17]:
def recursive_api_call(product_title, product_desc, product_cat, d):
    
    """ This function contains a recursive API call, every time going one category level deeper """
    
    # create API payload
    api_payload = create_prompt(product_title, product_desc, product_cat, d)
    
    # call API
    last_response = call_api(api_payload)
    
    # removes trailing dots which ChatGPT returns every now and then
    last_response = last_response.strip('.')
    
    # if a product falls in the Electronics category, the recursive API call starts one level deeper,
    # so within 'Computer', 'Keukenapparaten', 'Wearables', 'Persoonlijke verzorgingsapparaten', 'Witgoed' etc.
    if product_cat.startswith('Elektronica') and 'Elektronica' not in result:
        result.append('Elektronica')
        
    else:
        # appends to result
        result.append(last_response)
       
    if result[-1] in d and d[result[-1]]:
        recursive_api_call(product_title, product_desc, product_cat, d[result[-1]])

In [18]:
# loop over bestseller_lst and use some custom logic
final_result = []
number_of_products = 1000
counter = 0

for row in bestseller_lst:
    
    mdm_id = row[1]
    
    if counter > 0 and counter % 10 == 0:
        print(f'Now at {counter} or {round((counter / number_of_products) * 100, 1)}%')
    
    # break if limit is reached
    if counter == number_of_products - 1:
        print(row)
        break
    
    # if product is checked with AI before, skip over row
    elif mdm_id in already_recategorized_ids:
        continue
    
    # if mdm id not in product_dict (or current feed), skip for next time!
    elif not product_dict.get(mdm_id):
        continue
    
    else:    
        result = []
        product_title, product_desc, chunk_name, product_cat = product_dict[mdm_id]
        
        try:
            recursive_api_call(product_title, product_desc, product_cat, cat_dict)
            counter += 1
            
        except Exception as error:
            print("An error occurred:", error)
        
        result_str = " > ".join(result)
        final_result.append([mdm_id, f'https://www.blokker.nl/{mdm_id}.html', product_title, product_cat, chunk_name, result_str])

Now at 10 or 1.0%
Now at 10 or 1.0%
Now at 10 or 1.0%
Now at 10 or 1.0%
Now at 10 or 1.0%
Now at 20 or 2.0%
Now at 20 or 2.0%
Now at 20 or 2.0%
Now at 20 or 2.0%
Now at 20 or 2.0%
Now at 20 or 2.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
Now at 30 or 3.0%
x-ratelimit-remaining-requests: 499
Slept for 21 seconds (RT)
Now at 40 or 4.0%
Now at 40 or 4.0%
Now at 40 or 4.0%
Now at 40 or 4.0%
Now at 50 or 5.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 60 or 6.0%
Now at 70 or 7.0%
Now at 70 or 7.0%
Now at 70 or 7.0%
Now at 70 or 7.0%
No

In [20]:
%cd C:\\Users\\KoeReu\\Downloads

with open('openai_recategorization_resultset_gpt4o_mirakl_bestsellers_top_1000.csv', 'w', newline = '\n', encoding = 'utf-8') as f:
    csv_writer = csv.writer(f, delimiter = ';')
    csv_writer.writerow(['mdm_id', 'url', 'product_title', 'current_cat', 'current_chunk', 'ai_cat', 'new_chunk', 'equals'])
    for mdm_id, url, product_title, current_cat, current_chunk, ai_cat in final_result:
        
        # check if ai_cat (ChatGPT result) actually exists
        new_chunk = chunk_dict.get(ai_cat, False)
        
        # check if current cat equals ChatGPT result (AI category)
        eq = current_cat == ai_cat
        
        # write to csv
        csv_writer.writerow([mdm_id, url, product_title, current_cat, current_chunk, ai_cat, new_chunk, eq])

C:\Users\KoeReu\Downloads
